<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/model/Amazon_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
!pip install tensorflow_text

In [39]:
# nostri import 
import random
import timeit
import pickle


import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import tensorflow_text
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [40]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [41]:
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [42]:
df = pd.read_csv('/content/drive/Shareddrives/Deep Learning/datasets/Amazon/Reviews.csv')
print(f"COLONNE:{df.columns}")
print(f"DIMENSIONE:{df.shape}")

COLONNE:Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')
DIMENSIONE:(568454, 10)


In [43]:
del df['Id']
del df['ProductId']
del df['UserId']
del df['ProfileName']
del df['HelpfulnessNumerator']
del df['HelpfulnessDenominator']
del df['Time']

In [44]:
df['review'] = df['Summary']+df['Text']
del df['Summary']
del df['Text']
df.review.fillna("",inplace = True)
df.head()

,Score,review
0,5,Good Quality Dog FoodI have bought several of ...
1,1,Not as AdvertisedProduct arrived labeled as Ju...
2,4,"""Delight"" says it allThis is a confection that..."
3,2,Cough MedicineIf you are looking for the secre...
4,5,Great taffyGreat taffy at a great price. Ther...


In [45]:
df["review_type"] = df["Score"].apply(lambda x: "negative" if x < 4 else "positive")

In [46]:
df.review_type.value_counts()

positive    443777
negative    124677
Name: review_type, dtype: int64

In [47]:
positive_reviews = df[df.review_type == "positive"]
negative_reviews = df[df.review_type == "negative"]

In [48]:
positive_df = positive_reviews.sample(n=len(negative_reviews), random_state=RANDOM_SEED)
negative_df = negative_reviews

In [49]:
review_df = positive_df.append(negative_df).reset_index(drop=True)
review_df.shape

(249354, 3)

In [50]:
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [51]:
from sklearn.preprocessing import OneHotEncoder

type_one_hot = OneHotEncoder(sparse=False).fit_transform(
  review_df.review_type.to_numpy().reshape(-1, 1)
)

In [52]:
train_reviews, test_reviews, y_train, y_test =\
  train_test_split(
    review_df.review, 
    type_one_hot, 
    test_size=.1, 
    random_state=RANDOM_SEED
  )

In [53]:
X_train = []
for r in tqdm(train_reviews):
  emb = use(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_train.append(review_emb)

X_train = np.array(X_train)

100%|██████████| 224418/224418 [56:16<00:00, 66.46it/s]


In [54]:
X_test = []
for r in tqdm(test_reviews):
  emb = use(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_test.append(review_emb)

X_test = np.array(X_test)

100%|██████████| 24936/24936 [06:14<00:00, 66.51it/s]


In [55]:
with open("/content/drive/Shareddrives/Deep Learning/datasets/Amazon/X_train.pkl", 'wb') as output:
  pickle.dump(X_train, output, protocol=4)

with open("/content/drive/Shareddrives/Deep Learning/datasets/Amazon/X_test.pkl", 'wb') as output:
  pickle.dump(X_test, output, protocol=4)

In [56]:
with open("/content/drive/Shareddrives/Deep Learning/datasets/Amazon/X_train.pkl", 'rb') as output:
  print(output)
  X_train=pickle.load(output)

with open("/content/drive/Shareddrives/Deep Learning/datasets/Amazon/X_test.pkl", 'rb') as output:
  print(output)
  X_test=pickle.load(output)

<_io.BufferedReader name='/content/drive/Shareddrives/Deep Learning/datasets/Amazon/X_train.pkl'>
<_io.BufferedReader name='/content/drive/Shareddrives/Deep Learning/datasets/Amazon/X_test.pkl'>


In [57]:
X_test.shape

(24936, 512)

In [58]:
model = keras.Sequential()

model.add(keras.layers.Dense(units=256, input_shape=(X_train.shape[1], ), activation='relu'))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 164,482
Trainable params: 164,482
Non-trainable params: 0
_________________________________________________________________


In [59]:
history = model.fit(
    X_train, y_train, 
    epochs=15, 
    batch_size=16, 
    validation_split=0.1, 
    verbose=1, 
    shuffle=True
)

Epoch 1/15
12624/12624 [==============================] - 32s 3ms/step - loss: 0.3047 - accuracy: 0.8717 - val_loss: 0.2857 - val_accuracy: 0.8793
Epoch 2/15
12624/12624 [==============================] - 31s 2ms/step - loss: 0.2734 - accuracy: 0.8870 - val_loss: 0.2674 - val_accuracy: 0.8893
Epoch 3/15
12624/12624 [==============================] - 32s 2ms/step - loss: 0.2532 - accuracy: 0.8959 - val_loss: 0.2635 - val_accuracy: 0.8935
Epoch 4/15
12624/12624 [==============================] - 31s 2ms/step - loss: 0.2357 - accuracy: 0.9042 - val_loss: 0.2575 - val_accuracy: 0.8946
Epoch 5/15
12624/12624 [==============================] - 31s 2ms/step - loss: 0.2201 - accuracy: 0.9114 - val_loss: 0.2531 - val_accuracy: 0.8995
Epoch 6/15
12624/12624 [==============================] - 30s 2ms/step - loss: 0.2067 - accuracy: 0.9170 - val_loss: 0.2467 - val_accuracy: 0.9010
Epoch 7/15
12624/12624 [==============================] - 31s 2ms/step - loss: 0.1950 - accuracy: 0.9215 - val_loss: 0

In [60]:
train_reviews[180138]

'Smoky BaconThis bacon was really good and smoky.  It was not great, though.  I would shop around before ordering again.'

In [61]:
df.iloc[94752].review

'Made a neurotically picky brother happy.  Or as close to happy as he ever getsTo say that my brother is "picky" is the most extreme of understatements.  His finicky culinary mandates are the most outrageous, unreasonable and demanding of almost any human I\'ve ever known.  His love of nuts is extremely persnickety, and he has been known to turn his nose up at some of the most expensive gourmet nut selections available.<br />Sunflower Food and Spice Co Honey Toasted Cashews gave me a first ever accomplishment in my gift-giving history with my sibling.  This year, instead of the usual  "eh, not really that good actually."  or the one time in 1996 when I got a "meh, they\'re ok I guess."  He said this:<br /><br />"Best cashews I\'ve ever had."<br /><br />Truly, I can not tell you how significant a statement that is.'

In [62]:
review_df.iloc[94752]

Score                                                          5
review         Not CoffeeMy thinking about Teeccino is that i...
review_type                                             positive
Name: 94752, dtype: object

In [63]:
y_test[0]

array([0., 1.])

In [64]:
frase="s.  His finicky culinary mandates are the most outrageous, unreasonable and demanding of almost any human I\'ve ever known.  His love of nuts is extremely persnickety, and he has been known to turn his nose up at some of the most expensive gourmet nut selections available.<br />Sunflower Food and Spice Co Honey Toasted Cashews gave me a first ever accomplishment in my gift-giving history with my sibling.  This year, instead of the usual  eh, not really that good actually.  or the one time in 1996 when I got a meh, they\'re ok I guess.  He said this:<br /><br />Best cashews I\'ve ever had.<br /><br />Truly, I can not tell you how significant a statement that is."

emb_frase=use(frase)

model.predict(emb_frase)

array([[0.2918787 , 0.70812136]], dtype=float32)